<a href="https://colab.research.google.com/github/gregory-mc/twitter-streaming/blob/main/TStreaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TWITTER - API STREAMING
 (Python-Tweepy-Sqlite3) 

## Getting Tweets

In [ ]:
import tweepy
import csv
import dataset
import json
import datetime
import sqlite3

class TweetsListener(tweepy.StreamListener):

    def on_connect(self):
        print("I'm connected to API STREAMING!")

    def on_status(self, status):
      if (not status.retweeted) and ('RT @' not in status.text):

        user = status.user.name
        name = status.user.screen_name
        location = status.user.location
        created = status.created_at
        retweets = status.retweet_count
        likes=status.favorite_count
        try:
            text = status.extended_tweet["full_text"]
        except AttributeError:
            text = status.text
        entities=(created,user,text,name,location,retweets,likes)
        cursorObj.execute('''INSERT INTO tweets VALUES(?,?,?,?,?,?,?)''',entities)
        con.commit()
      return True


    def on_error(self, status_code):
        print("Error", status_code)




#Main.py or just Notebook file

try:

  # Puedes usar un archivo config o extraer de variables de entorno
  # You can use a config file or enviroment vars
  

  #     API KEY     #
  consumer_key = ""
  consumer_secret = ""
  access_token = ""
  access_token_secret = ""


  #     DATABASE     #
  con = sqlite3.connect("tweets.db") #database name
  cursorObj = con.cursor()
  cursorObj.execute("CREATE TABLE IF NOT EXISTS tweets(created date,user text, tweet text, user_name text,location text, retweets integer, likes integer)")
  con.commit()
  

  # AUTH API TWITTER #
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)
  api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, )


  #  API STREAMING   #
  stream = TweetsListener()
  streamingApi = tweepy.Stream(auth=api.auth, listener=stream)
  streamingApi.filter(track=['#DebatePresidencialJNE']) # Peruvian Theme

except KeyboardInterrupt: #I used Colab, so when I stopped it throwed an exception because i didn't use a  rate limit.
  print("Se ha recolectado: ", cursorObj.execute("SELECT count(*) FROM tweets").fetchall()[0][0])
  con.close()



Estoy conectado!
Se ha recolectado:  8


## Reading from database

In [ ]:
import pandas as pd
con = sqlite3.connect("tweets.db") #database name
a=pd.read_sql(con=con,sql="SELECT * FROM tweets")
a.head()

,created,user,tweet,user_name,location,retweets,likes
0,2021-03-29 20:33:33,danielgamboa02,#DebatePresidencialJNE Ya empezó la tendencia ...,danielgamboa021,None,0,0
1,2021-03-29 20:33:40,mar,"holis, voten por mi gatito porfas\n\nhttps://t...",marimar54_,None,0,0
2,2021-03-29 20:33:40,Gregory Morales,Prueba 3 #DebatePresidencialJNE,gmorales_c,None,0,0
3,2021-03-29 20:34:00,Waliq,#NoALaLeyTrans #debatepresidencialjne,Waliq76139295,None,0,0
4,2021-03-29 20:34:08,VITALIDAD Y LONGEVIDAD,¿Quién te preparó el meme? porque de inteligen...,rennazzer,Perú,0,0


In [ ]:
a

,created,user,tweet,user_name,location,retweets,likes
0,2021-03-29 20:33:33,danielgamboa02,#DebatePresidencialJNE Ya empezó la tendencia ...,danielgamboa021,None,0,0
1,2021-03-29 20:33:40,mar,"holis, voten por mi gatito porfas\n\nhttps://t...",marimar54_,None,0,0
2,2021-03-29 20:33:40,Gregory Morales,Prueba 3 #DebatePresidencialJNE,gmorales_c,None,0,0
3,2021-03-29 20:34:00,Waliq,#NoALaLeyTrans #debatepresidencialjne,Waliq76139295,None,0,0
4,2021-03-29 20:34:08,VITALIDAD Y LONGEVIDAD,¿Quién te preparó el meme? porque de inteligen...,rennazzer,Perú,0,0
5,2021-03-29 20:34:30,danielgamboa02,#DebatePresidencialJNE,danielgamboa021,None,0,0
6,2021-03-29 20:34:34,Un Gordito,¿Para el debate de hoy se compran piqueos y ce...,ungorditoart,None,0,0
7,2021-03-29 20:34:37,🦋 Flore 🦋,Espero que aquí haya seriedad por parte de los...,florebass,Lima-Peru,0,0


In [ ]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user            1 non-null      object
 1   tweet           1 non-null      object
 2   user_name       1 non-null      object
 3   user_followers  1 non-null      int64 
 4   created         1 non-null      object
 5   retweet_count   1 non-null      int64 
dtypes: int64(2), object(4)
memory usage: 176.0+ bytes


This file was made to get data for a study case.

# **References**

1.   [Tomas Reneboldi](https://tomasreneboldi.medium.com/c%C3%B3mo-obtener-datos-de-twitter-hacer-todo-esto-con-tweepy-3-5-21d60cd6e2c1)
2.   [Tutorial de Python SQLite3](https://likegeeks.com/es/tutorial-de-python-sqlite3/)

